<a href="https://colab.research.google.com/github/Sadiya-Akter-Mim/MovieLens-Recommender-Assignment/blob/main/DataSynthesis_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [ ]:
movies_url  = "https://raw.githubusercontent.com/Sadiya-Akter-Mim/MovieLens-Recommender-Assignment/main/movies.csv"
ratings_url = "https://raw.githubusercontent.com/Sadiya-Akter-Mim/MovieLens-Recommender-Assignment/main/ratings.csv"